In [ ]:
!python -m pip install --upgrade pip
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install opencv-python

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import cv2

In [ ]:
IMG_Size = 32
MIN_elevation = -50
MAX_elevation = 32

In [ ]:
__dir = "AllImages"
Datas = []

for dosya in os.listdir(__dir):
    dosya_adı = os.path.splitext(dosya)[0]  # Dosya uzantısını kaldırma
    dosya_p = dosya_adı.split('_')
    dosya_p[1] = re.sub(r'_?\d+', '', dosya_p[1])
    
    # Resmin türü ve şehir adını alıyoruz
    resim_turu = dosya_p[0]
    sehir = dosya_p[1]
    
    if(sehir == "ElevationMatrix"):
        sehir = resim_turu
        resim_turu = "ElevationMatrix"
    
    Datas.append({
        "Location" :  __dir + "/" + dosya,
        "City" : sehir,
        "Type" : resim_turu
    })

In [ ]:
Datas = pd.DataFrame(Datas)
Datas

In [ ]:
ElevationsMap = []
MinesMaps = []
RegionsMaps = []
GreenAreaMaps = []
LivestockMaps = []
AgricultralMaps = []

In [ ]:
CityNames = Datas["City"].unique().tolist()

for city in CityNames:
    MinesMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "MinesMap"), "Location"].values[0])
    RegionsMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "RegionsMap"), "Location"].values[0])
    GreenAreaMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "GreenAreaMap"), "Location"].values[0])
    LivestockMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "LivestockMap"), "Location"].values[0])
    AgricultralMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "AgricultralMap"), "Location"].values[0])
    ElevationsMap.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "ElevationMatrix"), "Location"].values[0])

del CityNames

In [ ]:
def ImgDivider(image, size=128):
    piece_width = size
    piece_height = size

    height, width = image.shape

    pieces = []
    for y in range(0, height, piece_height):
        for x in range(0, width, piece_width):
            piece_right = min(x + piece_width, width)
            piece_bottom = min(y + piece_height, height)
            piece_width_actual = piece_right - x
            piece_height_actual = piece_bottom - y
            
            piece = np.zeros((piece_height, piece_width), dtype=np.uint8)
            piece[:piece_height_actual, :piece_width_actual] = image[y:piece_bottom, x:piece_right]
            pieces.append(piece)

    return pieces

In [ ]:
def scale_array(array, scale_x, scale_y):
    height, width = array.shape
    
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)
    
    scaled_array = np.zeros((new_height, new_width), dtype=array.dtype)
    
    for i in range(new_height):
        for j in range(new_width):
            original_i = int(i / scale_y)
            original_j = int(j / scale_x)
            
            if original_i >= height or original_j >= width:
                scaled_array[i, j] = 0
            else:
                scaled_array[i, j] = array[original_i, original_j]
    
    return scaled_array

In [ ]:
def ElevationToImg(elevation, width, height):
    elevation_df = pd.DataFrame(elevation)
    
    max_x = max(elevation_df["x"]) + 1
    max_y = max(elevation_df["y"]) + 1
    
    array = np.zeros((max_y, max_x), dtype=int)
    
    for index, data in elevation_df.iterrows():
        x = data["x"]
        y = data["y"]
        elevation_val = data["elevation"]
        array[y, x] = elevation_val
        
    if max_x <= width and max_y <= height:
        scaleX = width / max_x
        scaleY = height / max_y
        resized_array = scale_array(array, scaleX, scaleY)
    else:
        resized_array = cv2.resize(array, (width, height), interpolation=cv2.INTER_AREA)

    return resized_array

In [ ]:
Images = {
    "ElevationMaps" : [],
    "AgricultralMaps" : [],
    "GreenAreaMaps" : [],
    "LivestockMaps" : [],
    "MinesMaps" : [],
    "RegionsMaps" : []
}

for index in range(len(RegionsMaps)):
    min_value = 0
    max_value = 1

    reg_img = cv2.imread(RegionsMaps[index], cv2.IMREAD_GRAYSCALE)
    reg_img = cv2.normalize(reg_img, None, min_value, max_value, cv2.NORM_MINMAX)

    min_img = cv2.imread(MinesMaps[index], cv2.IMREAD_GRAYSCALE)
    min_img = cv2.normalize(min_img, None, min_value, max_value, cv2.NORM_MINMAX)

    gre_img = cv2.imread(GreenAreaMaps[index], cv2.IMREAD_GRAYSCALE)
    gre_img = cv2.normalize(gre_img, None, min_value, max_value, cv2.NORM_MINMAX)

    liv_img = cv2.imread(LivestockMaps[index], cv2.IMREAD_GRAYSCALE)
    liv_img = cv2.normalize(liv_img, None, min_value, max_value, cv2.NORM_MINMAX)

    agr_img = cv2.imread(AgricultralMaps[index], cv2.IMREAD_GRAYSCALE)
    agr_img = cv2.normalize(agr_img, None, min_value, max_value, cv2.NORM_MINMAX)
    
    elv_img = cv2.imread(ElevationsMap[index], cv2.IMREAD_GRAYSCALE)
    elv_img = cv2.normalize(elv_img, None, min_value, max_value, cv2.NORM_MINMAX)
    
    imgSize = gre_img

    Images["RegionsMaps"].extend(ImgDivider(reg_img, IMG_Size))
    Images["MinesMaps"].extend(ImgDivider(min_img, IMG_Size))
    Images["GreenAreaMaps"].extend(ImgDivider(gre_img, IMG_Size))
    Images["LivestockMaps"].extend(ImgDivider(liv_img, IMG_Size))
    Images["AgricultralMaps"].extend(ImgDivider(agr_img, IMG_Size))
    Images["ElevationMaps"].extend(ImgDivider(elv_img, IMG_Size))

In [ ]:
deleteThisIndexes = []

for index in range(len(Images["AgricultralMaps"])):
    UV_reg = np.unique(Images["RegionsMaps"][index])
    UV_min = np.unique(Images["MinesMaps"][index])
    UV_gre = np.unique(Images["GreenAreaMaps"][index])
    UV_liv = np.unique(Images["LivestockMaps"][index])
    UV_agr = np.unique(Images["AgricultralMaps"][index])

    if(len(UV_reg) == 1 and len(UV_min) == 1 and len(UV_gre) == 1 and len(UV_liv) == 1 and len(UV_agr) == 1 and UV_reg[0] == 0):
        deleteThisIndexes.append(index)

In [ ]:
for key in Images.keys():
    image_list = Images[key]
    Images[key] = np.delete(image_list, deleteThisIndexes, axis=0)

In [ ]:
normalized_maps = np.copy(Images["ElevationMaps"])
normalized_maps[normalized_maps > MAX_elevation] = MAX_elevation
normalized_maps[normalized_maps < MIN_elevation] = MIN_elevation
normalized_maps = (normalized_maps - MIN_elevation) / (MAX_elevation - MIN_elevation)
normalized_maps[normalized_maps >= 1] = 1
normalized_maps[normalized_maps <= 0] = 0

Images["ElevationMaps"] = normalized_maps

In [ ]:
np.unique(Images["RegionsMaps"][5])

In [ ]:
ratios = []
for arrayIndex in range(len(Images["RegionsMaps"])):
    total_ones = np.sum(Images["RegionsMaps"][arrayIndex])
    total_elements = np.prod(Images["RegionsMaps"][arrayIndex].shape)
    ratio = total_ones / total_elements
    ratio = np.around(ratio, decimals=1)
    ratios.append(ratio)

Images["RegionsMaps"] = ratios

In [ ]:
inputsVals = np.stack((Images["AgricultralMaps"], Images["MinesMaps"], Images["GreenAreaMaps"], Images["LivestockMaps"], Images["ElevationMaps"]), axis=-1)

In [ ]:
outputMaps = (Images["RegionsMaps"])
outputMaps = np.array(outputMaps)

In [ ]:
del Images
del normalized_maps
del ratios
del Datas
del deleteThisIndexes
del ElevationsMap
del MinesMaps
del RegionsMaps
del GreenAreaMaps
del LivestockMaps
del AgricultralMaps

In [ ]:
outputMaps

# AI

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers

In [ ]:
input_shape = (IMG_Size, IMG_Size, 5)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(inputsVals, outputMaps, epochs=8)

In [ ]:
predictions = model.predict(inputsVals[:10])

In [ ]:
np.around(predictions, decimals=1)

In [ ]:
outputMaps[:10]

In [ ]:
model.save("Models/Fourth.keras")